In [1]:
import sys
import re
from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
import nltk.data
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import pickle
from collections import Iterable

from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI
import ner
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn import metrics
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

<ipython-input-1-650da48464a8>:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Iterable


In [3]:
%load_ext autoreload
%autoreload 2
from unwiki import unwiki

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# The results for the search for definition (currently just Wikipedia)
with open('/media/hd1/wikipedia/wiki_definitions_improved.txt', 'r') as wiki_f:
    wiki = wiki_f.readlines()

In [6]:
wiki[0]

'Arithmetic mean  -#-%-  Definition  -#-%-  \'The \\\'\\\'\\\'arithmetic mean\\\'\\\'\\\' (or \\\'\\\'\\\'mean\\\'\\\'\\\' or \\\'\\\'\\\'average\\\'\\\'\\\'), <math>\\\\bar{x}</math> (read <math>x</math> \\\'\\\'bar\\\'\\\'), is the mean of the <math>n</math> values <math>x_1,x_2,\\\\ldots,x_n</math>.<ref name="JM">{{cite book| last = Medhi| first = Jyotiprasad| title = Statistical Methods: An Introductory Text| url = https://books.google.com/?id=bRUwgf_q5RsC| year = 1992| publisher = New Age International| isbn = 9788122404197| pages = 53–58 }}</ref>\\n\\nThe arithmetic mean  is the most commonly used and readily understood measure of central tendency in a [[data set]]. In statistics, the term average refers to any of the measures of central tendency. The arithmetic mean of a set of observed data is defined as being equal to the sum of the numerical values of each and every observation divided by the total number of observations. Symbolically, if we have a data set consisting of the 

In [7]:
# Get data and train the Sentence tokenizer
# Uses a standard algorithm (Kiss-Strunk) for unsupervised sentence boundary detection
text = ''
for i in range(550):
    text += unwiki.loads(eval(wiki[i].split('-#-%-')[2]))

trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
tokenizer = PunktSentenceTokenizer(trainer.get_params())
print(tokenizer._params.abbrev_types)

{'j.w', 'eng', 'e.g', 'p.h.d', 'spacewalks', 'al', 'vibrations', 'neighbourhood', "'is", 'missions', 'r.a', 'ginebra', 'ton', 'juniper', 'jie', 'i.e', 'dr', 'cf', 'z-1', '2π', 'jr', 'ex', 's^2', 'mixture', 'hk', 'sow', 'u.n', 'etc', 'wings', 'az', 'x+2', 'ca', 'pl', 'u.s'}


In [8]:
# The cleaning up of the wiki markup so that it looks like normal written english
title, section, defin = wiki[850].split('-#-%-')
dclean = unwiki.loads(eval(defin))
print(title)
print(dclean)
defin

Indicator function  
The indicator function of a subset A of a set X is a function

_display_math_

defined as

_display_math_

The Iverson bracket allows the equivalent notation, _inline_math_, to be used instead of _inline_math_.

The function _inline_math_ is sometimes denoted _inline_math_, _inline_math_, _inline_math_ or even just _inline_math_. (The Greek letter _inline_math_ appears because it is the initial letter of the Greek word χαρακτήρ, which is the ultimate origin of the word characteristic.)

The set of all indicator functions on _inline_math_ can be identified with _inline_math_, the power set of _inline_math_.  Consequently, both sets are sometimes denoted by _inline_math_. This is a special case (_inline_math_) of the notation _inline_math_ for the set of all functions _inline_math_.


'  "The indicator function of a subset \'\'A\'\' of a set \'\'X\'\' is a function\\n\\n:<math>\\\\mathbf{1}_A \\\\colon X \\\\to \\\\{ 0,1 \\\\} </math>\\n\\ndefined as\\n\\n:<math>\\\\mathbf{1}_A(x) :=\\n\\\\begin{cases}\\n1 &\\\\text{if } x \\\\in A, \\\\\\\\\\n0 &\\\\text{if } x \\\\notin A.\\n\\\\end{cases}\\n</math>\\n\\nThe [[Iverson bracket]] allows the equivalent notation, <math>[x\\\\in A]</math>, to be used instead of <math>\\\\mathbf{1}_A(x)</math>.\\n\\nThe function <math>\\\\mathbf{1}_A</math> is sometimes denoted <math>I_A</math>, <math>\\\\chi_A</math>, \'\'K<sub>A</sub>\'\' or even just <math>A</math>. (The [[Greek alphabet|Greek letter]] <math>\\\\chi</math> appears because it is the initial letter of the Greek word χαρακτήρ, which is the ultimate origin of the word \'\'characteristic\'\'.)\\n\\nThe set of all indicator functions on <math>X</math> can be identified with <math>\\\\mathcal{P}(X)</math>, the [[power set]] of <math>X</math>.  Consequently, both sets are so

In [36]:
# Get the data and POS and NER tags for each definition (LONG TIME)
def_lst = []
for i in range(len(wiki)):
    try:
        title, section, defin_raw = wiki[i].split('-#-%-')
        defin_all = unwiki.loads(eval(defin_raw))
        for d in tokenizer.tokenize(defin_all):
            if title.lower().strip() in d.lower():
                pos_tokens = pos_tag(word_tokenize(d))
                def_ner = ner.bio_tag.bio_tagger(title.strip().split(), pos_tokens)
                other_ner = [((d[0],d[1]),d[2]) for d in def_ner]
                tmp_dict = {'title': title,
                           'section': section,
                           'defin': d,
                           'ner': other_ner}
                def_lst.append(tmp_dict)
    except ValueError:
        print('parsing error')

In [38]:
def_lst[0]

{'title': 'Arithmetic mean  ',
 'section': '  Definition  ',
 'defin': 'The arithmetic mean (or mean or average), _inline_math_ (read _inline_math_ bar), is the mean of the _inline_math_ values _inline_math_.',
 'ner': [(('The', 'DT'), 'O'),
  (('arithmetic', 'JJ'), 'B-DFNDUM'),
  (('mean', 'NN'), 'I-DFNDUM'),
  (('(', '('), 'O'),
  (('or', 'CC'), 'O'),
  (('mean', 'VB'), 'O'),
  (('or', 'CC'), 'O'),
  (('average', 'VB'), 'O'),
  ((')', ')'), 'O'),
  ((',', ','), 'O'),
  (('_inline_math_', 'NNP'), 'O'),
  (('(', '('), 'O'),
  (('read', 'VB'), 'O'),
  (('_inline_math_', 'NNP'), 'O'),
  (('bar', 'NN'), 'O'),
  ((')', ')'), 'O'),
  ((',', ','), 'O'),
  (('is', 'VBZ'), 'O'),
  (('the', 'DT'), 'O'),
  (('mean', 'NN'), 'O'),
  (('of', 'IN'), 'O'),
  (('the', 'DT'), 'O'),
  (('_inline_math_', 'NN'), 'O'),
  (('values', 'VBZ'), 'O'),
  (('_inline_math_', 'NNP'), 'O'),
  (('.', '.'), 'O')]}

In [26]:
class NamedEntityChunker(ChunkParserI):
    def __init__(self, train_sents, **kwargs):
        assert isinstance(train_sents, Iterable)
 
        self.feature_detector = features
        self.tagger = ClassifierBasedTagger(
            train=train_sents,
            feature_detector=features,
            **kwargs)
    def parse(self, tagged_sent):
        chunks = self.tagger.tag(tagged_sent)
 
        # Transform the result from [((w1, t1), iob1), ...] 
        # to the preferred list of triplets format [(w1, t1, iob1), ...]
        iob_triplets = [(w, t, c) for ((w, t), c) in chunks]
 
        # Transform the list of triplets to nltk.Tree format
        return conlltags2tree(iob_triplets)       
        
random.shuffle(def_lst)
training_samples = [d['ner'] for d in def_lst[:int(len(def_lst) * 0.9)]]
test_samples = [d['ner'] for d in def_lst[int(len(def_lst) * 0.9):]]
 
print("#training samples = %s" % len(training_samples) )   # training samples = 55809
print("#test samples = %s" % len(test_samples))            # test samples = 6201
 

#training samples = 12602
#test samples = 1401


In [29]:
#train the NER Chunking Classifier (TAKES A LONG TIME)
%time chunker = NamedEntityChunker(training_samples)

CPU times: user 15.3 s, sys: 96.7 ms, total: 15.4 s
Wall time: 15.4 s


In [30]:
# Evaluate the most common metrics on the test dataset
unpack = lambda l: [(tok, pos, ner) for ((tok, pos), ner) in l]
Tree_lst = [conlltags2tree(unpack(t)) for t in test_samples]
print(chunker.evaluate(Tree_lst))

ChunkParse score:
    IOB Accuracy:  91.3%%
    Precision:     31.5%%
    Recall:        68.4%%
    F-Measure:     43.2%%


### Other Scores

Training with an amount of the dataset and evaluating with the rest
* With 80% of the dataset

* 60% of the data

```ChunkParse score:
    IOB Accuracy:  91.0%%
    Precision:     30.7%%
    Recall:        63.9%%
    F-Measure:     41.5%%```
    
```ChunkParse score:
    IOB Accuracy:  90.6%%
    Precision:     32.4%%
    Recall:        68.7%%
    F-Measure:     44.0%%```

* 90% of the data

```ChunkParse score:
    IOB Accuracy:  91.2%%
    Precision:     32.0%%
    Recall:        68.0%%
    F-Measure:     43.5%%```
    


In [39]:
# An example of a user fed definition
print(chunker.parse(pos_tag(word_tokenize("We define a Banach space as a complete vector space."))))

(S
  We/PRP
  define/VBP
  a/DT
  (DFNDUM Banach/NNP space/NN)
  as/IN
  a/DT
  complete/JJ
  vector/NN
  space/NN
  ./.)


In [40]:
def prepare_for_metrics(int_range, chunker_fn, data_set = test_samples, print_output=False):
    '''
    `int_range` is an integer range
    NEEDS A TEST_SAMPLES VARIABLE CREATED WHEN SPLITTING THE 
    TRAINING AND TESTING DATA
    Returns two vectors ready to be used in the 
    metrics classification function
    '''
    if isinstance(int_range, int):
        int_range = [int_range]
    y_true = []
    y_pred = []
    for i in int_range:
        sample = data_set[i]
        sm = [s[0] for s in sample]
        y_true_tmp = [s[1] for s in sample]
        predicted = [v[2] for v in tree2conlltags(chunker_fn.parse(sm))]
        y_true += y_true_tmp
        y_pred += predicted
        if print_output:
            for k,s in enumerate(sm):
                print('{:15} {:>10}  {:>10}'.format(s[0], y_true_tmp[k], predicted[k]))
    return y_true, y_pred

In [63]:
OO = prepare_for_metrics(119, chunker, data_set=test_samples, print_output=True)

The                      O           O
table                    O    B-DFNDUM
below                    O    I-DFNDUM
enumerates               O           O
the                      O           O
order             B-DFNDUM           O
of                I-DFNDUM           O
magnitude         I-DFNDUM           O
of                       O           O
some                     O           O
numbers                  O           O
in                       O           O
light                    O           O
of                       O           O
this                     O           O
definition               O           O
:                        O           O
The                      O           O
geometric                O    B-DFNDUM
mean                     O    I-DFNDUM
of                       O           O
_inline_math_            O           O
and                      O           O
_inline_math_            O           O
is                       O           O
_inline_math_            

In [49]:
y_true, predicted = prepare_for_metrics(range(len(test_samples)), chunker)
print(metrics.classification_report(y_true, predicted))

              precision    recall  f1-score   support

    B-DFNDUM       0.35      0.74      0.47      1351
    I-DFNDUM       0.27      0.80      0.40      1055
           O       0.99      0.91      0.95     44583

   micro avg       0.90      0.90      0.90     46989
   macro avg       0.53      0.82      0.61     46989
weighted avg       0.95      0.90      0.92     46989



In [28]:
def features(tokens, index, history):
    """
    `tokens`  = a POS-tagged sentence [(w1, t1), ...]
    `index`   = the index of the token we want to extract features for
    `history` = the previous predicted IOB tags
    """
 
    # init the stemmer
    stemmer = SnowballStemmer('english')
 
    # Pad the sequence with placeholders
    tokens = [('[START3]', '[START3]'),('[START2]', '[START2]'), ('[START1]', '[START1]')] +\
    list(tokens) + [('[END1]', '[END1]'), ('[END2]', '[END2]'), ('[END3]', '[END3]')]
    history = ['[START3]', '[START2]', '[START1]'] + list(history)
 
    # shift the index with 3, to accommodate the padding
    index += 3
 
    word, pos = tokens[index]
    prevword, prevpos = tokens[index - 1]
    prevprevword, prevprevpos = tokens[index - 2]
    prev3word, prev3pos = tokens[index - 3]
    nextword, nextpos = tokens[index + 1]
    nextnextword, nextnextpos = tokens[index + 2]
    next3word, next3pos = tokens[index + 3]
    previob = history[index - 1]
    prevpreviob = history[index - 2]
    prev3iob = history[index - 3]
    contains_dash = '-' in word
    contains_dot = '.' in word
    allascii = all([True for c in word if c in string.ascii_lowercase])
 
    allcaps = word == word.capitalize()
    capitalized = word[0] in string.ascii_uppercase
 
    prevallcaps = prevword == prevword.capitalize()
    prevcapitalized = prevword[0] in string.ascii_uppercase
 
    nextallcaps = prevword == prevword.capitalize()
    nextcapitalized = prevword[0] in string.ascii_uppercase
    
    is_math = lambda w:(w == '_inline_math_') or (w == '_display_math_')
    ismath = is_math(word)
    isprevmath = is_math(prevword)
    isprevprevmath = is_math(prevprevword)
 
    return {
        'word': word,
        'lemma': stemmer.stem(word),
                'pos': pos,
        'all-ascii': allascii,
 
        'next-word': nextword,
        'next-lemma': stemmer.stem(nextword),
        'next-pos': nextpos,
 
        'next-next-word': nextnextword,
        'nextnextpos': nextnextpos,
 
        'next3word': next3word,
        'next3pos': next3pos,
        
        'prev-word': prevword,
        'prev-lemma': stemmer.stem(prevword),
        'prev-pos': prevpos,
 
        'prev-prev-word': prevprevword,
        'prev-prev-pos': prevprevpos,
 
        'prev3word': prev3word,
        'prev3pos': prev3pos,
        
        'prev-iob': previob,
        
        'prev-prev-iob': prevpreviob,
 
        'contains-dash': contains_dash,
        'contains-dot': contains_dot,
 
        'all-caps': allcaps,
        'capitalized': capitalized,
 
        'prev-all-caps': prevallcaps,
        'prev-capitalized': prevcapitalized,
 
        'next-all-caps': nextallcaps,
        'next-capitalized': nextcapitalized,
        
        'ismath': ismath,
        'isprevmath': isprevmath,
        'isprevprevmath': isprevprevmath,
    }
 